In [33]:
import os
import pandas as pd
import numpy as np
import io
import subprocess

## 1. Concatenate fasta sequences of selected chromosomes

The sequences are expected to be in a one file, if they are not combine them at first using step 0

### Step 0

In [28]:
def combine_fasta(output_file, *fasta_files):
    with open(output_file, 'w') as out_f:
        for idx, f in enumerate(fasta_files) :
            with open(f, 'r') as in_file:
                out_f.write(in_file.read())
                out_f.write('\n')
    

In [29]:
combine_fasta('combined_fasta', 'first.fa', 'second.fa')

### Step 1

In [4]:
def concat_fasta(input_file, output_prefix, chromosomes: list):
    select_chr = []
    cur_chr = None
    with open(input_file, 'r') as inp_file:
        buf = io.StringIO()
        for line in inp_file:
            line = line.strip()
            if line.startswith('>'):
                chr_name = line.lstrip('>').split(' ')[0]
                if chr_name in chromosomes:
                    cur_chr = chr_name
                    select_chr.append(chr_name)
                else:
                    cur_chr = None
            else:
                if cur_chr is not None:
                    buf.write(line)

    combined_seqs = buf.getvalue()
    length_genome = len(combined_seqs)
    select_chr_str = '_'.join(select_chr)

    with open(output_prefix, 'w') as out_pref:
        out_pref.write(f">concatenated_{select_chr_str} LN:{length_genome}\n{combined_seqs}")
    
    output_file = f'{output_prefix}_concatenated_chromosomes.fa'
    try:
        os.rename(output_prefix, output_file)
        
    except FileNotFoundError:
        print(f'Error: file {out_pref} is not found')

In [5]:
input_file = 'GRCh38_full_analysis_set_plus_decoy_hla.fa'
output_file = 'combined_chr1_chr2_test'
concat_fasta(input_file, output_file, ('chr1', 'chr2'))

## 2. Merge VCF files

 0. Before you merge VCF files you need to find the length of each chromosome in order to shift your SNPs. I'll be using combined file from step 0

In [30]:
#write only chromosomes name without > in input
def get_length(input_file, chromosomes: list):
    sel_chr = []
    chr_lengths = []
    with open(input_file, 'r') as inp_file:
        cur_chr = None
        # first_line = inp_file.readline()
        # cur_chr = first_line.strip().lstrip('>').split(' ')[0]
        for line in inp_file:
            line = line.strip()
            if line.startswith('>'):
                if cur_chr is not None:
                    chr_lengths.append((cur_chr, len(buf1.getvalue())))
                chr_name = line.lstrip('>').split(' ')[0]
                if  chr_name in chromosomes:
                    sel_chr.append(chr_name)
                    cur_chr = chr_name
                    buf1 = io.StringIO()
                else:
                    cur_chr = None
                    buf1 = None
            else:
                if cur_chr is not None:
                    buf1.write(line)
        if cur_chr is not None:
            chr_lengths.append((cur_chr, len(buf1.getvalue())))

    return chr_lengths

In [32]:
input_file = 'GRCh38_full_analysis_set_plus_decoy_hla.fa'
print(get_length(input_file, ['chr1', 'chr2']))

[('chr1', 248956422), ('chr2', 242193529)]


1. Run these functions in the exact order in order get concatenated vcf file in the end

In [ ]:
def sort_vcf(input_file, output_file):
    subprocess.run([f'bcftools sort {input_file} -o {output_file}.vcf'], shell=True) #sort
    
def bgzip_vcf(input_file):
    subprocess.run([f'bgzip {input_file}'], shell=True)

def shift_positions_one_chr(input_file, output_file, shift: int): #you can run it for each chr file independently and then merge
    subprocess.run([f"bcftools view {input_file} | awk 'BEGIN{{OFS='\t'}} /^#/ {{print; next}} {{ $2 = $2 + {shift}; print}}' > "])

def rename_chr(input_file, output_file, change_names): #change_names.txt must include in the first column old names and in the second new names, sep=' '
    subprocess.run([f'bcftools annotate --rename-chrs {change_names} {input_file} -Oz -o {output_file}.gz'])

def concat_vcf(output_file, *vcf_files):
    subprocess.run([f'bcftools concat --naive {vcf_files} -o {output_file}'])

## 3. Change positions in your recombination map

In this step you need to shift the positions inside your recombination map, add first line '1   0.0' and add recombination point between two chromosomes with recombination rate 0.5 (to point that these parts are independent). Also ensue that your positions are of int type and rates in float type and the positions go in the ascending order. Unfortunately, SLiM uses the same recombination rates for both sexes (or maybe I missed sth), so I will use recombination map for females from this source https://www.nature.com/articles/s41586-024-08450-5

chr1 0
chr2 500 #for length of the first chromosome in the 1-based system
chr3 600
chr4 700

In [ ]:
def shift_recomb(recomb_map, shifts, chromosomes): #for the first chromosome shift is zero  
    df = pd.read_csv(recomb_map, skiprows=10, sep='\t')
    cur_shift = 0
    inter_chr_break = []
    for idx, chr in enumerate(chromosomes):
        shift = int(shifts[idx].split(' ')[1])
        cur_shift = shift + cur_shift
        mask = (df['Chr']== chr)
        df.loc[mask, ['pos']] = df.loc[mask, ['pos']] + shift #add length of the first chromosome
        if idx > 0:
            inter_chr_break.append([[shift, 0.5], [shift + 1, 0.5]])

    rec_pos_chr1_chr2_temp = df.loc[:, ['pos', 'cMperMb']]

    zero_row = pd.DataFrame([[1, 0.0]], columns=['pos', 'cMperMb'])

    inter_chr_break_df = pd.DataFrame(inter_chr_break, columns=['pos', 'cMperMb']) #in 1 based system

    rec_pos_chr1_chr2 = pd.concat([zero_row, rec_pos_chr1_chr2_temp, inter_chr_break_df], axis=0)
    rec_pos_chr1_chr2['pos'].astype(int)
    rec_pos_chr1_chr2['cMperMb'].astype(float)
    rec_pos_chr1_chr2 = rec_pos_chr1_chr2.sort_values(by='pos')

    rec_pos_chr1_chr2.to_csv(f'{recomb_map}.tsv', sep='\t', header=None, index=None) #no header and no indexes must be included

## 4. Run simulation in the same way as for one chromosome

Better do it in bash. You must have your simulated pedigree structure and at least one parent for each descendant except for founders. Example code:

python py_ped_sim_init/run_ped_sim.py -t sim_genomes_exact -n gen_ped.nx -e founders.txt -v first_family_chr1_chr2_sorted_concatenated_chr2_shifted_renamed_slim_fil.vcf.gz -f combined_chr1_chr2.fa -rm recomb_pos_mat_chr1_chr2.tsv -o first_family_simul_two_chr

## 5. Change chromosomes names for SNP positions in a vcf file with simulated SNPs 

[[chr1, 500]
[chr2, 600]
[chr3, 700]]

In [ ]:
def shift_back(input_file, chr_lengths): #chr_lengths - list
    output_file = f"{input_file}_shifted_back.vcf"
    open(output_file, 'w').close()

    prev_point = 0
    for chr_line in chr_lengths: #shift positions for each chromosome except the first
        chr = chr_line[0]
        
        end = int(chr_line[1]) + prev_point
        subprocess.run(f"awk -v prev={prev_point} -v end={end} -v chrname={chr}"
                    f"'BEGIN{{OFS=\"\\t\"}} /^#/ {{print; next}} "
                    f" ($2 > prev && $2 <= end) {{$1=chr; $2=$2-prev}} 1' "
                    f"{input_file}' >> {input_file}_shifted_back.vcf", shell=True)